# 📘 Notebook: Ewaluacja modeli ML – AI Warsztaty

🧱 Spis treści

1. Wprowadzenie – po co ewaluować modele?

2. Klasyfikacja

3. Regresja

4. Grupowanie (unsupervised)

5. Sieci neuronowe (specyfika ewaluacji)

6. Walidacja modeli i overfitting

7. Cheat Sheet – skróty, wybór metryk

##1. 🧠 Po co ewaluować modele?

### Czy Twój model działa dobrze?

- Ewaluacja modelu to nie tylko sprawdzenie "czy działa", ale:
  - Czy uczy się właściwego wzorca?
  - Czy generalizuje?
  - Czy jest lepszy niż losowe zgadywanie?

🔎 Dzięki ewaluacji:
- porównujemy modele i wybieramy najlepszy
- unikamy pułapek typu overfitting
- dobieramy odpowiednie metody do konkretnego problemu


##2. 📊 Klasyfikacja

### Co to jest klasyfikacja?

To przewidywanie jednej z kilku klas (np. TAK/NIE, kot/pies, pozytywny/negatywny).

📌 Przykłady:
- Email to spam / nie spam
- Czy pacjent ma raka
- Czy klient spłaci kredyt

---

### 📏 Metryki klasyfikacyjne

1. **Accuracy** – % poprawnych odpowiedzi
   - Działa tylko przy zrównoważonych klasach
   - Nie sprawdza się przy danych typu: 95% klasy "NIE"

2. **Precision** – spośród przewidzianych jako pozytywne, ile było faktycznie pozytywnych  
   > Wysoka precision: mało fałszywych alarmów

3. **Recall** – spośród faktycznie pozytywnych, ile model wykrył  
   > Wysoki recall: mało przypadków przeoczonych

4. **F1-score** – balans precision i recall

5. **AUC-ROC** – ocena zdolności odróżnienia klas (niezależnie od progu)

---

### 📌 Kiedy używać której?

| Problem | Ważna metryka |
|--------|----------------|
| Zbalansowane dane | Accuracy, F1 |
| Niezbalansowane dane | F1, Precision, Recall |
| Bezpieczeństwo, zdrowie | Recall |
| Blokowanie spamu | Precision |
| Wynik to prawdopodobieństwo | AUC-ROC |


### Ćwiczenie 1: Wytrenuj klasyfikator na zbiorze breast_cancer

1) Naucz się obliczać accuracy, precision, recall i F1.

In [18]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
print(data.target_names)
print(set(data.target))

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, stratify=data.target)

model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


['malignant' 'benign']
{np.int64(0), np.int64(1)}
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        42
           1       0.97      0.94      0.96        72

    accuracy                           0.95       114
   macro avg       0.94      0.95      0.94       114
weighted avg       0.95      0.95      0.95       114



}### Ćwiczenie 2: Zinterpretuj wyniki

1) Kiedy model bardziej myli się (fałszywe pozytywy czy negatywy)?

2) Co to może znaczyć w kontekście diagnozy medycznej?

##3. 📈 Regresja

### Czym jest regresja?

Model przewiduje wartość liczbową:
- prognoza cen
- przewidywanie temperatury
- szacowanie ryzyka

---

### 📏 Metryki regresji

1. **MAE (Mean Absolute Error)**  
   - Średni błąd bezwzględny
   - Intuicyjny: "średnio się pomyliłem o X"

2. **MSE (Mean Squared Error)**  
   - Bardziej karze duże błędy

3. **RMSE (Root MSE)**  
   - MSE znormalizowane – ta sama jednostka co target

4. **R² (coefficient of determination)**  
   - Wskaźnik, jak dobrze model tłumaczy zmienność danych (0–1)

---

### 📌 Kiedy używać której?

| Problem | Metryka |
|--------|---------|
| Błąd ma być "ludzki" | MAE |
| Chcesz karać duże błędy | MSE / RMSE |
| Chcesz ocenić ogólną jakość dopasowania | R² |


### Ćwiczenie 1: Porównanie MAE i RMSE

1) Zobacz różnicę między karaniem dużych błędów.

In [19]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAE:", mae)
print("RMSE:", rmse)


MAE: 0.535360732232198
RMSE: 0.7343661735836567


### Ćwiczenie 2: Stwórz wykres błędów

1) Narysuj histogram błędów predykcji (y_test - y_pred)

2) Co możesz z niego wyczytać?

##4. 🔍 Grupowanie (Clustering)

### Brak etykiet – jak ocenić?

W grupowaniu (np. KMeans) nie mamy "prawdziwych" odpowiedzi.  
Model sam szuka podobieństw – musimy ocenić jakość klastrów.

---

### 📏 Metryki unsupervised:

1. **Silhouette Score**  
   - Miara spójności i separacji klas  
   - od -1 do 1 (im wyżej, tym lepiej)

2. **Calinski-Harabasz Index**  
   - Stosunek odległości między klastrami do wewnętrznej spójności  
   - im wyższy, tym lepiej

3. **Davies-Bouldin Index**  
   - Niższy = lepiej

---

### 📌 Co robić?

- Próbować różnych K i porównywać metryki
- Wizualizować: PCA, t-SNE


### Ćwiczenie 1: KMeans + silhouette score

1) Naucz się oceniać jakość klastrów

In [15]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

X, _ = make_blobs(n_samples=300, centers=4, random_state=42)

kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X)

score = silhouette_score(X, labels)
print("Silhouette Score:", score)


Silhouette Score: 0.7915830011443038


### Ćwiczenie 2: Zmień liczbę klastrów

1) Sprawdź, jak zmienia się wynik silhouette_score

2) Czy zawsze więcej klastrów = lepiej?

## 5. 🧠 Sieci neuronowe

### Sieci uczą się na wiele epok – trzeba kontrolować postęp

📏 Monitorujemy:
- **Loss (funkcja straty)** na train i validation
- Accuracy/F1 na validation
- Early stopping – zatrzymaj trening gdy val_loss rośnie

📊 Często wykresy:
- loss vs epoch
- val_loss vs epoch

Używamy callbacków np. w TensorFlow:
```python
EarlyStopping(monitor='val_loss', patience=3)


### Ćwiczenie 1: Ucz sieć na MNIST i rysuj loss

1) Zobacz overfitting i użycie early stopping

In [16]:
!pip install tensorflow

In [17]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten`
from tensorflow.keras.callbacks import EarlyStopping

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks=[es])


KeyboardInterrupt: 

### Ćwiczenie 2: Wykres loss i accuracy

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.title('Loss vs Val Loss')
plt.show()


## 6. 🧪 Walidacja i overfitting

### Podstawy:

- **Train/test split** – dzielimy dane na trening i test
- **Cross-validation** – np. K-fold
- **StratifiedKFold** – zachowuje proporcje klas

📌 Overfitting = model za bardzo uczy się danych treningowych  
➡️ Działa świetnie na train, źle na test

### Jak to rozpoznać?

- accuracy na train: 99%, na test: 60% → źle
- rosnący train_loss, rosnący val_loss → źle

### Ćwiczenie 1: Cross-validation w klasyfikacji

In [ ]:
print("X_train shape:", X_train.shape)
print("X_train ndim:", X_train.ndim)

Twoje dane mają kształt (60000, 28, 28), czyli 60 tys. obrazków 28x28 pikseli – wygląda na klasyczny zbiór MNIST albo coś podobnego. To są dane obrazowe, a RandomForestClassifier nie działa bezpośrednio na obrazach 2D – trzeba je najpierw spłaszczyć do postaci 2D.

In [ ]:
# Spłaszczenie danych: (60000, 28, 28) → (60000, 784)
X_train_flat = X_train.reshape(X_train.shape[0], -1)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')

print("F1-scores:", scores)
print("Średni F1:", scores.mean())


### Ćwiczenie 2: StratifiedKFold na niezbalansowanym zbiorze

1) Zrób eksperyment: weź zbiór z niezbalansowanymi klasami i porównaj zwykły KFold vs StratifiedKFold.

2) Jakie są różnice w wynikach?

## 7. 🧠 Cheat Sheet – metryki

| Problem         | Metryki                   |
|----------------|---------------------------|
| Klasyfikacja   | Accuracy, F1, Precision, Recall, AUC |
| Regresja       | MAE, MSE, RMSE, R²         |
| Clustering     | Silhouette, Calinski-Harabasz, Davies-Bouldin |
| Sieci neuronowe | Loss, val_loss, val_acc     |


# 8. 💼 Projekt – przykładowa ewaluacja

Zobacz, że wyżej,: ten sam zbiór danych, ten sam problem, zrobiłeś za pomocą różnych modeli, o których mówiliśmy na warsztatach. Możesz zobaczyć, że analiza tematu i to w jaki sposób podejdziesz do tematu jest kluczowym elementem każdego projektu.

## 📘 Dane: np. predykcja choroby (klasyfikacja binarna)

1. Wczytaj dane (z CSV)
2. Podziel na train/test
3. Trenuj np. Logistic Regression
4. Oblicz metryki: accuracy, precision, recall, F1
5. Dodaj wykresy (np. confusion matrix)
6. BONUS: zrób cross-validation i porównaj metryki

📎 Tip: możesz użyć `classification_report` ze sklearn
